In [5]:
#Data Manipulation
import numpy as np
import pandas as pd

#Plotting and visualization
import matplotlib.pyplot as plt
import seaborn as sns #Some shiny visualizations

#Utilities
import json
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, f1_score, recall_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


#Artificial Neural Networks
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Activation, Dropout, BatchNormalization

Using TensorFlow backend.


In [10]:
#--------- Project Administration ------------
DatasetFilepath = './GeneticSimulation/CORE_SLE_RA_Control_blood_panels.csv'
#--------------------------------------------

In [11]:
data = pd.read_csv(DatasetFilepath)
data = data.drop(data.columns[0], axis=1)

In [12]:
if False:
    # Create correlation matrix
    corr_matrix = data.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # Find features with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

    # Drop features 
    data.drop(to_drop, axis=1, inplace=True)
    data.to_csv('CORE_SLE_RA_Control_blood_panels.csv')

In [13]:
label = LabelEncoder()
int_data = label.fit_transform(data['Condition'])
int_data = int_data.reshape(len(int_data), 1)

onehot_data = OneHotEncoder(sparse=False)
onehot_data = onehot_data.fit_transform(int_data)

y = onehot_data

In [14]:
x = data.drop(['Name', 'Condition'], axis=1)

In [15]:
# Prepare and scale the data. 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=25)

x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)

In [16]:
#--------------------------------------------------------------------------
def make_that_model(layers, loss_function, optimizer, learning_rate, exit_activation=None):
    model = keras.Sequential()
    for layer in layers:
        model.add(getattr(keras.layers, layer["name"])(**layer["options"]))

    if exit_activation!=None:
        model.add(Activation(exit_activation))

    op = getattr(keras.optimizers, optimizer)(learning_rate=(learning_rate))
    model.compile(loss=loss_function, optimizer=op, metrics=['accuracy'])

    return model
#--------------------------------------------------------------------------
def model_create(parameterization, load_file=None):

  model = make_that_model(
      parameterization.get('layers'),
      parameterization.get('loss_function'),
      parameterization.get('optimizer'),
      parameterization.get('learning_rate'),
      )
  
  if load_file != None:
    model.load_weights(load_file)
  
  return model
#--------------------------------------------------------------------------

In [28]:
config = {
    "layers": [
      {
        "name": "BatchNormalization",
        "options": {
          "input_shape": [
            15979
          ]
        }
      },
      {
        "name": "BatchNormalization",
        "options": {}
      },
      {
        "name": "BatchNormalization",
        "options": {}
      },
      {
        "name": "BatchNormalization",
        "options": {}
      },
      {
        "name": "Dense",
        "options": {
          "units": 121,
          "activation": "elu",
          "use_bias": True
        }
      },
      {
        "name": "Dense",
        "options": {
          "units": 4,
          "activation": "softmax"
        }
      }
    ],
    "loss_function": "binary_crossentropy",
    "optimizer": "Nadam",
    "batch_size": 64,
    "learning_rate": 0.04867806312636558,
  }

In [29]:
model = model_create(config)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 15979)             63916     
_________________________________________________________________
batch_normalization_4 (Batch (None, 15979)             63916     
_________________________________________________________________
batch_normalization_5 (Batch (None, 15979)             63916     
_________________________________________________________________
batch_normalization_6 (Batch (None, 15979)             63916     
_________________________________________________________________
dense_2 (Dense)              (None, 121)               1933580   
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 488       
Total params: 2,189,732
Trainable params: 2,061,900
Non-trainable params: 127,832
______________________________________

In [38]:
hist = model.fit(
        x=x_train, y=y_train, 
        validation_data=(x_test,y_test),
        validation_freq=5,
#         workers=8,
#         use_multiprocessing=True,
        epochs=200, 
        verbose=1,
        )

Epoch 1/200
2/2 [==============================] - 0s 104ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/200
2/2 [==============================] - 0s 63ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/200
2/2 [==============================] - 0s 94ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/200
2/2 [==============================] - 0s 100ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/200
2/2 [==============================] - 1s 394ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/200
2/2 [==============================] - 0s 63ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/200
2/2 [==============================] - 0s 70ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/200
2/2 [==============================] - 0s 85ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/200
2/2 [==============================] - 0s 98ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/200
2/2 [===============

2/2 [==============================] - 0s 34ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 74/200
2/2 [==============================] - 0s 54ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 75/200
2/2 [==============================] - 0s 102ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 76/200
2/2 [==============================] - 0s 49ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 77/200
2/2 [==============================] - 0s 39ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 78/200
2/2 [==============================] - 0s 42ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 79/200
2/2 [==============================] - 0s 37ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 80/200
2/2 [==============================] - 0s 110ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 81/200
2/2 [==============================] - 0s 39ms/step - loss: 0.0000e+00 - accuracy:

2/2 [==============================] - 0s 144ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 146/200
2/2 [==============================] - 0s 61ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 147/200
2/2 [==============================] - 0s 48ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 148/200
2/2 [==============================] - 0s 41ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 149/200
2/2 [==============================] - 0s 58ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 150/200
2/2 [==============================] - 0s 120ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 151/200
2/2 [==============================] - 0s 43ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 152/200
2/2 [==============================] - 0s 58ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 153/200
2/2 [==============================] - 0s 43ms/step - loss: 0.0000e+00 - a

In [39]:
pred = model.predict(x_test)

In [40]:
batch_size=30


In [41]:
stat = classification_report(np.argmax(pred,-1), np.argmax(y_test, -1), output_dict=True)
metrics = dict(
        zip(model.metrics_names,
            model.evaluate(x=x_test, y=y_test, batch_size=batch_size)))

1/1 [==============================] - 1s 680ms/step - loss: 0.0000e+00 - accuracy: 1.0000


In [42]:
metrics

{'loss': 0.0, 'accuracy': 1.0}

In [43]:
stat

{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 4},
 '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 5},
 '3': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3},
 'accuracy': 1.0,
 'macro avg': {'precision': 1.0,
  'recall': 1.0,
  'f1-score': 1.0,
  'support': 12},
 'weighted avg': {'precision': 1.0,
  'recall': 1.0,
  'f1-score': 1.0,
  'support': 12}}

In [44]:
label.inverse_transform(np.argmax(pred, axis=-1))

array(['disease: Rheumatoid Arthiritis (DMARD-IR)',
       'disease: Rheumatoid Arthiritis (DMARD-IR)',
       'disease: Rheumatoid Arthiritis (DMARD-IR)',
       'disease: SLE (Systemic LUPUS Erythomatosus)',
       'disease: Rheumatoid Arthiritis (DMARD-IR)', 'disease: Control',
       'disease: SLE (Systemic LUPUS Erythomatosus)',
       'disease: SLE (Systemic LUPUS Erythomatosus)', 'disease: Control',
       'disease: Control', 'disease: Control',
       'disease: Rheumatoid Arthiritis (DMARD-IR)'], dtype=object)

In [45]:
label.inverse_transform(np.argmax(y_test, axis=-1))

array(['disease: Rheumatoid Arthiritis (DMARD-IR)',
       'disease: Rheumatoid Arthiritis (DMARD-IR)',
       'disease: Rheumatoid Arthiritis (DMARD-IR)',
       'disease: SLE (Systemic LUPUS Erythomatosus)',
       'disease: Rheumatoid Arthiritis (DMARD-IR)', 'disease: Control',
       'disease: SLE (Systemic LUPUS Erythomatosus)',
       'disease: SLE (Systemic LUPUS Erythomatosus)', 'disease: Control',
       'disease: Control', 'disease: Control',
       'disease: Rheumatoid Arthiritis (DMARD-IR)'], dtype=object)